<a href="https://colab.research.google.com/github/tam8738/NghiViec/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

def check_data_for_logistic_regression_from_file(file_path, label_column, file_type='csv', separator=','):
    """
    Kiểm tra dữ liệu từ file xem có phù hợp để train hồi quy logistic hay không.

    Parameters:
    - file_path: Đường dẫn đến file dữ liệu.
    - label_column: Tên cột chứa nhãn.
    - file_type: Loại file ('csv', 'excel').
    - separator: Dấu phân tách trong file (mặc định: ',').

    Returns:
    - Báo cáo các vấn đề (nếu có) và gợi ý xử lý.
    """
    # 1. Đọc dữ liệu từ file
    print(f"Đang đọc file: {file_path}")
    try:
        if file_type == 'csv':
            df = pd.read_csv(file_path, sep=separator)
        elif file_type == 'excel':
            df = pd.read_excel(file_path)
        else:
            raise ValueError("file_type không được hỗ trợ. Chọn 'csv' hoặc 'excel'.")
    except FileNotFoundError:
        print(f"LỖI: Không tìm thấy file tại {file_path}.")
        return
    except Exception as e:
        print(f"LỖI khi đọc file: {e}")
        return

    # 2. Kiểm tra cột nhãn
    if label_column not in df.columns:
        print(f"LỖI: Cột nhãn '{label_column}' không tồn tại trong file.")
        print("Các cột hiện có:", df.columns.tolist())
        return

    # 3. Tách đặc trưng và nhãn
    X = df.drop(columns=[label_column])
    y = df[label_column]

    print(f"\nSố lượng mẫu: {len(df)}")
    print(f"Số đặc trưng: {len(X.columns)}")
    print(f"Tên đặc trưng: {X.columns.tolist()}")

    # 4. Kiểm tra dữ liệu
    print("\n=== KIỂM TRA DỮ LIỆU CHO HỒI QUY LOGISTIC ===")

    # Kiểm tra giá trị thiếu
    missing_values = X.isnull().sum()
    if missing_values.any():
        print("CẢNH BÁO: Tồn tại giá trị thiếu trong các đặc trưng:")
        print(missing_values[missing_values > 0])
        print("Gợi ý: Xử lý giá trị thiếu bằng cách điền giá trị trung bình, trung vị hoặc xóa dòng.")
    else:
        print("OK: Không có giá trị thiếu trong đặc trưng.")

    if y.isnull().sum() > 0:
        print("CẢNH BÁO: Tồn tại giá trị thiếu trong nhãn.")
        print("Gợi ý: Xóa các dòng có nhãn thiếu hoặc gán giá trị mặc định.")
    else:
        print("OK: Không có giá trị thiếu trong nhãn.")

    # Kiểm tra kiểu dữ liệu
    non_numeric_cols = X.select_dtypes(exclude=['float64', 'int64']).columns
    if len(non_numeric_cols) > 0:
        print("CẢNH BÁO: Tồn tại cột không phải kiểu số:", non_numeric_cols.tolist())
        print("Gợi ý: Chuyển đổi cột này thành dạng số (ví dụ: mã hóa one-hot cho biến phân loại).")
    else:
        print("OK: Tất cả đặc trưng đều là kiểu số.")

    # Kiểm tra nhãn nhị phân
    unique_labels = np.unique(y.dropna())
    if len(unique_labels) != 2:
        print(f"CẢNH BÁO: Nhãn không phải nhị phân. Số giá trị duy nhất: {len(unique_labels)}")
        print("Gợi ý: Đảm bảo nhãn chỉ có 2 giá trị (0/1 hoặc True/False).")
    else:
        print("OK: Nhãn là nhị phân.")

    # Kiểm tra giá trị vô cực hoặc NaN
    if np.any(np.isinf(X.select_dtypes(include=['float64', 'int64']))) or np.any(np.isnan(X.select_dtypes(include=['float64', 'int64']))):
        print("CẢNH BÁO: Tồn tại giá trị vô cực hoặc NaN trong đặc trưng số.")
        print("Gợi ý: Thay thế hoặc xóa các giá trị này.")
    else:
        print("OK: Không có giá trị vô cực hoặc NaN trong đặc trưng số.")

    # Kiểm tra tỷ lệ nhãn
    label_counts = y.value_counts(normalize=True)
    print("\nTỷ lệ nhãn:")
    print(label_counts)
    if any(label_counts < 0.1):
        print("CẢNH BÁO: Dữ liệu mất cân bằng nghiêm trọng.")
        print("Gợi ý: Sử dụng kỹ thuật như oversampling, undersampling hoặc class-weight.")

    # Kiểm tra phạm vi giá trị đặc trưng
    print("\nPhạm vi giá trị của các đặc trưng số:")
    numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
    if len(numeric_cols) > 0:
        print(X[numeric_cols].describe().loc[['min', 'max']])
        if (X[numeric_cols].max() - X[numeric_cols].min()).max() > 1000:
            print("CẢNH BÁO: Một số đặc trưng có phạm vi giá trị lớn.")
            print("Gợi ý: Chuẩn hóa dữ liệu bằng StandardScaler hoặc MinMaxScaler.")
    else:
        print("Không có đặc trưng số để kiểm tra phạm vi.")

    print("\n=== KẾT THÚC KIỂM TRA ===")

# Sử dụng
if __name__ == "__main__":
    file_path = "bodulieu.csv"  # Đường dẫn đến file
    label_column = "danghiviec"  # Tên cột nhãn
    file_type = "csv"           # Loại file
    separator = ";"             # Dấu phân tách

    check_data_for_logistic_regression_from_file(file_path, label_column, file_type,separator)

Đang đọc file: bodulieu.csv

Số lượng mẫu: 14999
Số đặc trưng: 9
Tên đặc trưng: ['mucdohailong', 'danhgia_gannhat', 'soluong_duanthamgia', 'sogiolamtrungbinh_hangthang', 'sonamlamviec', 'tainanlaodong', 'duocthangchuc_trong5nam', 'phongban', 'mucluong']

=== KIỂM TRA DỮ LIỆU CHO HỒI QUY LOGISTIC ===
OK: Không có giá trị thiếu trong đặc trưng.
OK: Không có giá trị thiếu trong nhãn.
CẢNH BÁO: Tồn tại cột không phải kiểu số: ['phongban', 'mucluong']
Gợi ý: Chuyển đổi cột này thành dạng số (ví dụ: mã hóa one-hot cho biến phân loại).
OK: Nhãn là nhị phân.
OK: Không có giá trị vô cực hoặc NaN trong đặc trưng số.

Tỷ lệ nhãn:
danghiviec
0    0.761917
1    0.238083
Name: proportion, dtype: float64

Phạm vi giá trị của các đặc trưng số:
     mucdohailong  danhgia_gannhat  soluong_duanthamgia  \
min          0.09             0.36                  2.0   
max          1.00             1.00                  7.0   

     sogiolamtrungbinh_hangthang  sonamlamviec  tainanlaodong  \
min                

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
import joblib

# 1. Đọc dữ liệu
df = pd.read_csv('bodulieu.csv', sep=';')

# 2. Tiền xử lý
mucluong_map = {'low': 0, 'medium': 1, 'high': 2}
df['mucluong'] = df['mucluong'].map(mucluong_map)
df = pd.get_dummies(df, columns=['phongban'], drop_first=True)

# Loại bỏ cột 'tainanlaodong'
df = df.drop(columns=['tainanlaodong'])

# 3. Tách đặc trưng và nhãn
X = df.drop('danghiviec', axis=1)
y = df['danghiviec']

# 4. Chuẩn hóa và cân bằng
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# 5. Huấn luyện mô hình
model = LogisticRegression(class_weight='balanced', max_iter=1000)
model.fit(X_resampled, y_resampled)

# 6. Lưu mô hình và scaler
joblib.dump(model, 'logistic_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(X.columns.tolist(), 'feature_names.pkl')
print("✅ Mô hình và scaler đã được lưu thành công (đã loại bỏ 'tainanlaodong').")


Mô hình và scaler đã được lưu thành công (đã loại bỏ 'tainanlaodong').


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Đánh giá mô hình trên tập huấn luyện
y_pred = model.predict(X_resampled)
y_proba = model.predict_proba(X_resampled)[:, 1]

print("=== ĐÁNH GIÁ MÔ HÌNH ===")
print("Accuracy :", accuracy_score(y_resampled, y_pred))
print("Precision:", precision_score(y_resampled, y_pred))
print("Recall   :", recall_score(y_resampled, y_pred))
print("F1 Score :", f1_score(y_resampled, y_pred))
print("ROC AUC  :", roc_auc_score(y_resampled, y_proba))


=== ĐÁNH GIÁ MÔ HÌNH ===
Accuracy : 0.7664070703535176
Precision: 0.7532229892705647
Recall   : 0.7924396219810991
F1 Score : 0.7723338023964863
ROC AUC  : 0.8198377181284686
